In [1]:
import torch
import json
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, TrainingArguments, Trainer
from torch.utils.data import Dataset, DataLoader

In [11]:
class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=512):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer(
            text,
            return_tensors='pt',
            max_length=self.max_length,
            truncation=True,
            padding='max_length'
        )
        inputs = {key: val.squeeze(0) for key, val in inputs.items()}
        return inputs


In [4]:
# Training on Alex's text data
with open(r"E:\Projects\StardewScrapingSLM(FT_RAG)\data_gathering\assets\quotes_data\bcl_quotes.json") as f:
    data = json.load(f)
data = data['Alex']

In [ ]:
# pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-1B")

Device set to use cpu


In [ ]:
tokenizer = LlamaTokenizer.from_pretrained('llama-3.2-1b')
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

In [7]:
inputs = tokenizer(data, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [8]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=inputs['input_ids'],
)

In [10]:
# Fine-tune the model
trainer.train()

ValueError: The batch received was empty, your model won't be able to train on it. Double-check that your training dataset contains keys expected by the model: input_ids,attention_mask,position_ids,past_key_values,inputs_embeds,labels,use_cache,output_attentions,output_hidden_states,return_dict,cache_position,num_logits_to_keep,kwargs,label_ids,labels,label.

In [14]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import LlamaForCausalLM, LlamaTokenizer, Trainer, TrainingArguments

# Custom Dataset class
class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=512):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer(
            text,
            return_tensors='pt',
            max_length=self.max_length,
            truncation=True,
            padding='max_length'
        )
        inputs = {key: val.squeeze(0) for key, val in inputs.items()}
        return inputs


# Load the tokenizer and model
tokenizer = LlamaTokenizer.from_pretrained('llama-3.2-1b')

# Add a new special token for padding
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Load the model
model = LlamaForCausalLM.from_pretrained('llama-3.2-1b')
model.resize_token_embeddings(len(tokenizer))

# Create the dataset and dataloader
dataset = TextDataset(texts, tokenizer)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Fine-tune the model
trainer.train()

ImportError: 
LlamaTokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.
